# Use PCA to Orient a Randomly Rotated Object Up and Down Along It's Longest Axis

In [ ]:
import open3d as o3d
import numpy as np
# from zipfile import ZipFile
import gzip
import requests
from io import BytesIO
import copy
import os
from random import randint

In [ ]:
PLY_URL = "https://graphics.stanford.edu/data/3Dscanrep/xyzrgb/xyzrgb_statuette.ply.gz"

In [ ]:
PLY_LOCAL_PATH = "./statuette.ply"
if not os.path.exists(PLY_LOCAL_PATH):
    content = BytesIO(requests.get(PLY_URL).content)
    with gzip.open(content, "r") as gz:
        ply = gz.read()
    with open(PLY_LOCAL_PATH, "wb+") as f:
        f.write(ply)

In [ ]:
print("Load a ply point cloud, print it, and render it")
pcd = o3d.io.read_point_cloud(PLY_LOCAL_PATH)
# print(pcd)
# print(np.asarray(pcd.points))
# o3d.visualization.draw_geometries([pcd],
#                                   zoom=0.3412,
#                                   front=[0.4257, -0.2125, -0.8795],
#                                   lookat=[2.6172, 2.0475, 1.532],
#                                   up=[-0.0694, -0.9768, 0.2024])

In [ ]:
# o3d.visualization.draw_geometries([pcd])
# alpha = 1
# print(f"alpha={alpha:.3f}")

# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
#     pcd, alpha)
# mesh.compute_vertex_normals()
# o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
pcd_random_rotated = copy.deepcopy(pcd)
x_rot = np.pi / randint(1,8)
y_rot = np.pi / randint(1,8)
z_rot = np.pi / randint(1,8)
R = pcd_random_rotated.get_rotation_matrix_from_xyz((x_rot, y_rot, z_rot))
pcd_random_rotated.rotate(R, center=(0, 0, 0))

In [ ]:
pcd_random_rotated.paint_uniform_color(np.array([1, 0, 0]))

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=3)

In [ ]:
points = np.asarray(pcd.points)

In [ ]:
points[:5]

In [ ]:
out = pca.fit_transform(points)

In [ ]:
out[:5]

In [ ]:
out_pcd = o3d.geometry.PointCloud()
out_pcd.points = o3d.utility.Vector3dVector(out)

In [ ]:
out_rotated = copy.deepcopy(out_pcd)
R = out_rotated.get_rotation_matrix_from_xyz((0, 0, np.pi/2))
out_rotated.rotate(R, center=(0, 0, 0))
out_rotated.paint_uniform_color(np.array([0, 1, 0]))


In [ ]:
o3d.visualization.draw_geometries([pcd_random_rotated, out_rotated])

## Now Use Rotated as Reference For Global Registration and Iterative Closest Point Transformation to Correct the Original Random Rotated Object

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0, 0])
    target_temp.paint_uniform_color([0, 1, 0])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                    #   zoom=0.4459,
                                    #   front=[0.9288, -0.2951, -0.2242],
                                    #   lookat=[1.6784, 2.0612, 1.4451],
                                    #   up=[-0.3402, -0.9189, -0.1996]
                                      )

In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [ ]:
def prepare_dataset(source, target, voxel_size):
    print(":: Load two point clouds and disturb initial pose.")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [ ]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True, distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500))
    
    # result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    #     source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
    #     o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 4, [
    #         o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
    #         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
    #             distance_threshold)
    #     ], o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

In [ ]:
voxel_size = 0.05 # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source = pcd_random_rotated, target=out_rotated, voxel_size=voxel_size)

In [ ]:
# result_ransac = execute_global_registration(source_down, target_down,
#                                             source_fpfh, target_fpfh,
#                                             voxel_size)
# print(result_ransac)
# draw_registration_result(source_down, target_down, result_ransac.transformation)

In [ ]:
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [ ]:
result_ransac_fast = execute_fast_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac_fast)
draw_registration_result(source_down, target_down, result_ransac_fast.transformation)

In [ ]:
threshold = 0.025
# trans_init = np.asarray([[1, 0, 0, 0],
#                          [0, 1, 0, 0],
#                          [0, 0, 1, 0], 
#                          [0.0, 0.0, 0.0, 1.0]])
trans_init = result_ransac_fast.transformation

In [ ]:
trans_init

In [ ]:
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    pcd_random_rotated, out_rotated, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(pcd_random_rotated, out_rotated, reg_p2p.transformation)